# Get api key from 
https://aistudio.google.com/app/apikey

# Course url
https://learn.nvidia.com/courses/course-detail?course_id=course-v1:DLI+S-FX-15+V1

# Printing the models

In [5]:
# Direct Google Gemini API Usage
import google.generativeai as genai

for m in genai.list_models():
    if "generateContent" in m.supported_generation_methods:
        print(m.name)

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924


# Printing the embedders of gemini

In [6]:
for m in genai.list_models():
    if "embedContent" in m.supported_generation_methods:
        print(m.name)

models/embedding-001
models/text-embedding-004


# Getting information about a model

In [7]:
model_info = genai.get_model("models/embedding-001")
print(model_info)

Model(name='models/embedding-001',
      base_model_id='',
      version='001',
      display_name='Embedding 001',
      description='Obtain a distributed representation of a text.',
      input_token_limit=2048,
      output_token_limit=1,
      supported_generation_methods=['embedContent'],
      temperature=None,
      max_temperature=None,
      top_p=None,
      top_k=None)


# Reading the pdf

In [19]:
import PyPDF2

# Open the PDF file
pdf_path = 'herman-melville-moby-dick.pdf'
with open(pdf_path, 'rb') as file:
    reader = PyPDF2.PdfReader(file)
    text = ''
    for page_num in range(len(reader.pages)):
        page = reader.pages[page_num]
        text += page.extract_text()


# Splitting the book to chuncks and summarize them

In [27]:
from langchain_google_genai import GoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from typing import List,Tuple
import time
import os
from tenacity import (retry,stop_after_attempt,wait_fixed,retry_if_exception_type,before_sleep_log)
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def get_wait_time(retry_count: int) -> float:
    """Return wait time based on retry count"""
    if retry_count <= 3:
        return 4.0
    elif retry_count == 4:
        return 8.0
    else:
        return 10.0

class CustomWaitStrategy:
    """Custom wait strategy that follows the exact pattern from error messages"""
    def __init__(self):
        self.retry_count = 0

    def __call__(self, retry_state):
        self.retry_count += 1
        wait_time = get_wait_time(self.retry_count)
        return wait_time

# Set your API key
def init_google_api(api_key: str):
    """Initialize Google API key"""
    os.environ["GOOGLE_API_KEY"] = api_key

@retry(
    wait=CustomWaitStrategy(),
    stop=stop_after_attempt(5),
    retry=retry_if_exception_type(Exception),
    before_sleep=before_sleep_log(logger, logging.INFO)
)
def summarize_chunk(chunk: str, llm) -> str:
    """Summarize text using Gemini with exact retry timing"""
    try:
        prompt = f"""Please summarize the following text concisely in no more than 2-3 sentences:
        
        {chunk}
        
        Summary:"""
        
        return llm.invoke(prompt)
    except Exception as e:
        logger.error(f"Error during summarization: {str(e)}")
        raise

@retry(
    wait=CustomWaitStrategy(),
    stop=stop_after_attempt(5),
    retry=retry_if_exception_type(Exception),
    before_sleep=before_sleep_log(logger, logging.INFO)
)
def get_embedding(text: str, embeddings_model) -> List[float]:
    """Get embeddings using Google's embedding model with exact retry timing"""
    try:
        return embeddings_model.embed_query(text)
    except Exception as e:
        logger.error(f"Error during embedding: {str(e)}")
        raise

def process_text_chunks(chunks: List[str], api_key: str, batch_size: int = 2) -> tuple:
    """Process text chunks with conservative batching"""
    # Initialize API key
    init_google_api(api_key)
    
    # Initialize models
    llm = GoogleGenerativeAI(
        model="gemini-pro",  # Correct model name format
        temperature=0,
        top_p=0.8,
        google_api_key=api_key,
        max_output_tokens=750
    )
    
    embeddings_model = GoogleGenerativeAIEmbeddings(
        model="models/embedding-001",
        google_api_key=api_key
    )
    
    summaries = []
    embeddings = []
    
    # Process in batches
    for i in range(0, len(chunks), batch_size):
        batch = chunks[i:i + batch_size]
        logger.info(f"\nProcessing batch {i//batch_size + 1}/{(len(chunks) + batch_size - 1)//batch_size}")
        
        # Generate summaries for batch
        for chunk in batch:
            logger.info("Summarizing chunk...")
            summary = summarize_chunk(chunk, llm)
            summaries.append(summary)
            # Add mandatory wait between API calls
            time.sleep(4.0)
        
        # Generate embeddings for batch
        for summary in summaries[i:i + batch_size]:
            logger.info("Generating embedding...")
            embedding = get_embedding(summary, embeddings_model)
            embeddings.append(embedding)
            # Add mandatory wait between API calls
            time.sleep(4.0)
        
        # Add longer wait between batches
        if i + batch_size < len(chunks):
            logger.info("Waiting between batches...")
            time.sleep(8.0)
    
    return summaries, embeddings

def chunk_text(text: str, max_chunk_size: int = 750) -> List[str]:
    """Split text into chunks of appropriate size"""
    words = text.split()
    chunks = []
    current_chunk = []
    current_size = 0
    
    for word in words:
        word_size = len(word) + 1  # +1 for space
        if current_size + word_size > max_chunk_size:
            chunks.append(' '.join(current_chunk))
            current_chunk = [word]
            current_size = word_size
        else:
            current_chunk.append(word)
            current_size += word_size
    
    if current_chunk:
        chunks.append(' '.join(current_chunk))
    
    return chunks
# Your Google API key
API_KEY = "YOUR_API_KEY"
long_text = text
try:
    # Create chunks with specified size
    chunks = chunk_text(long_text, max_chunk_size=15000)
    # Process the chunks
    summaries, embeddings = process_text_chunks(
        chunks=chunks,
        api_key=API_KEY,
        batch_size=2
    )
    # Print results
    for i, (summary, embedding) in enumerate(zip(summaries, embeddings)):
        print(f"\nChunk {i+1} Summary:")
        print(summary)
        print(f"Embedding dimension: {len(embedding)}")   
except Exception as e:
    logger.error(f"An error occurred: {str(e)}")

INFO:__main__:
Processing batch 1/37
INFO:__main__:Summarizing chunk...
INFO:__main__:Summarizing chunk...
INFO:__main__:Generating embedding...
INFO:__main__:Generating embedding...
INFO:__main__:Waiting between batches...
INFO:__main__:
Processing batch 2/37
INFO:__main__:Summarizing chunk...
INFO:__main__:Summarizing chunk...
INFO:__main__:Generating embedding...
INFO:__main__:Generating embedding...
INFO:__main__:Waiting between batches...
INFO:__main__:
Processing batch 3/37
INFO:__main__:Summarizing chunk...
INFO:__main__:Summarizing chunk...
INFO:__main__:Generating embedding...
INFO:__main__:Generating embedding...
INFO:__main__:Waiting between batches...
INFO:__main__:
Processing batch 4/37
INFO:__main__:Summarizing chunk...
INFO:__main__:Summarizing chunk...
INFO:__main__:Generating embedding...
INFO:__main__:Generating embedding...
INFO:__main__:Waiting between batches...
INFO:__main__:
Processing batch 5/37
INFO:__main__:Summarizing chunk...
INFO:__main__:Summarizing chunk.


Chunk 1 Summary:
Herman Melville's "Moby-Dick" is a sprawling novel that follows the obsessive quest of Captain Ahab to hunt down the elusive white whale, Moby Dick. The narrative is narrated by Ishmael, a sailor who joins Ahab's crew on the whaling ship Pequod. The novel explores themes of obsession, revenge, and the human condition, and features a vast cast of characters and vivid descriptions of whaling life.
Embedding dimension: 768

Chunk 2 Summary:
The narrator, Ishmael, embarks on a whaling voyage to escape the mundane and embrace the wonders of the sea. He arrives in New Bedford and finds lodging at the dilapidated "Spouter-Inn," where he encounters a mysterious painting depicting a whale impaling itself on a ship's masts during a hurricane. The inn's interior is adorned with macabre whaling artifacts, including a whale's jawbone serving as a bar.
Embedding dimension: 768

Chunk 3 Summary:
At a dismal inn, the narrator encounters a group of rowdy sailors, including a mysteriou

# Saving the embaddings and the summaries

In [31]:
import faiss
import numpy as np
import json

# Convert embeddings to a numpy array
embedding_dim = len(embeddings[0])
embedding_matrix = np.array(embeddings).astype('float32')

# Initialize FAISS index
index = faiss.IndexFlatL2(embedding_dim)

# Add embeddings to FAISS index
index.add(embedding_matrix)

# Save the index to disk
faiss.write_index(index, 'faiss_book_index.index')

# Save summaries to a JSON file
with open('summaries.json', 'w') as f:
    json.dump(summaries, f)


# Processing queries

In [96]:
class DocumentRetrievalSystem:
    def __init__(
        self, 
        faiss_index_path: str,
        summaries_path: str,
        gemini_api_key: str,
        embedding_model: str = "models/embedding-001",
        num_results: int = 3
    ):
        """
        Initialize the retrieval system with necessary components.
        
        Args:
            faiss_index_path: Path to saved FAISS index
            summaries_path: Path to saved summaries JSON
            gemini_api_key: API key for Google Gemini
            embedding_model: Model to use for query embedding
            num_results: Number of relevant documents to retrieve
        """
        # Load FAISS index
        self.index = faiss.read_index(faiss_index_path)
        
        # Load summaries
        with open(summaries_path, 'r') as f:
            self.summaries = json.load(f)
            

        
        self.model = GoogleGenerativeAI(
        model="models/gemini-1.5-flash",  # Correct model name format

        google_api_key=gemini_api_key
    )
    
        self.embedding_model = GoogleGenerativeAIEmbeddings(
            model="models/embedding-001",
            google_api_key=gemini_api_key
        )
    
        
        
        self.num_results = num_results

    def get_query_embedding(self, query: str) -> np.ndarray:
        """Generate embedding for the query text."""
        embedding = self.embedding_model.embed_query(
            query,
            task_type="retrieval_query"
        )
        return np.array(embedding).astype('float32').reshape(1, -1)

    def retrieve_relevant_documents(
        self, 
        query_embedding: np.ndarray
    ) -> Tuple[List[str], List[float]]:
        """
        Retrieve most relevant documents using FAISS.
        
        Returns:
            Tuple of (list of relevant summaries, list of distances)
        """
        # Search the index
        distances, indices = self.index.search(query_embedding, self.num_results)
        
        # Get corresponding summaries
        relevant_summaries = [self.summaries[idx] for idx in indices[0]]
        
        return relevant_summaries, distances[0]

    def format_context(
        self, 
        summaries: List[str], 
        distances: List[float]
    ) -> str:
        """Format retrieved documents into context for the LLM."""
        context_pieces = []
        for i, (summary, distance) in enumerate(zip(summaries, distances)):
            relevance_score = 1 / (1 + distance)  # Convert distance to similarity score
            context_pieces.append(
                f"Document {i+1} (Relevance: {relevance_score:.2f}):\n{summary}\n"
            )
        return "\n".join(context_pieces)

    def generate_response(
        self, 
        query: str, 
        context: str
    ) -> str:
        """Generate response using Gemini based on query and context."""
        prompt = f"""
        Based on the following context, please answer the user's question.
        Be specific and only use information from the provided context.
        If the context doesn't contain relevant information, please say so.

        Context:
        {context}

        User Question: {query}
        """
        print(context,'\n\n\n\n')
        print(query)
        
        response = self.model.invoke(prompt)
        return response

    def process_query(self, query: str) -> str:
        """
        Main method to process a user query and return a response.
        
        Args:
            query: User's question
            
        Returns:
            Generated response based on retrieved relevant documents
        """
        try:
            # Get query embedding
            query_embedding = self.get_query_embedding(query)
            
            # Retrieve relevant documents
            relevant_docs, distances = self.retrieve_relevant_documents(query_embedding)
            
            # Format context
            context = self.format_context(relevant_docs, distances)
            
            # Generate response
            response = self.generate_response(query, context)
            
            return response
            
        except Exception as e:
            return f"An error occurred: {str(e)}"

# Initialize the system
retrieval_system = DocumentRetrievalSystem(
    faiss_index_path='faiss_book_index.index',
    summaries_path='summaries.json',
    gemini_api_key=API_KEY
)

query = "who is Captain Ahab  ?"
response = retrieval_system.process_query(query)
print(response)




Document 1 (Relevance: 0.70):
Captain Ahab, the enigmatic and formidable captain of the Pequod, is described as a complex and enigmatic figure. Despite his reputation as a skilled whaler and a stern leader, he is also known for his obsessive pursuit of the white whale, Moby Dick, which cost him his leg. While some crew members express concern about his mental state, others remain loyal, recognizing his determination and leadership qualities.

Document 2 (Relevance: 0.69):
Rachel's search for her missing children led her to find another orphan. This book was adapted from the original text obtained from the Gutenberg Project and typeset using XETEX.

Document 3 (Relevance: 0.68):
In the whaling industry, the captain stands while being rowed, unlike other ships where they sit comfortably. The Town-Ho, a whaling ship, encountered the Pequod and shared news of Moby Dick, including a mysterious and ominous incident involving the captain and a crew member. This secret, known only to a few sai